In [1]:
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import plotly.express as px

# This statement allow to display plot without asking to 
%matplotlib inline

# always make it pretty
plt.style.use('ggplot')

#warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore", category=FutureWarning)

## Loading the data

In [3]:
df = pd.read_excel('data/dataset_tasacion_train_vf.xlsx')
df.head()

,Fecha entrega del Informe,Tipo de vía,Piso,Departamento,Provincia,Distrito,Número de estacionamiento,Depósitos,Latitud (Decimal),Longitud (Decimal),Categoría del bien,Posición,Número de frentes,Edad,Elevador,Estado de conservación,Método Representado,Área Terreno,Área Construcción,Valor comercial (USD)
0,2020-02-16,1.0,NaN,Piura,Piura,Veintiseis de Octubre,NaN,NaN,-5.163182,-80.682388,Vivienda Unifamiliar,NaN,NaN,0.0,NaN,En construcción,Costos o reposición (directo),62.50,27.58,25278
1,2019-05-21,1.0,NaN,Lima,Lima,Surquillo,1.0,0.0,-12.115286,-77.003768,Departamento,NaN,NaN,1.0,NaN,Bueno,Comparación de mercado (directo),9.32,57.47,110807
2,2020-04-11,1.0,NaN,Junín,Satipo,Satipo,0.0,0.0,-11.235446,-74.629221,Vivienda Unifamiliar,NaN,NaN,7.0,NaN,Regular,Costos o reposición (directo),126.00,84.00,42285
3,2016-05-24,1.0,NaN,Lima,Lima,Santiago de Surco,0.0,0.0,-12.126384,-76.975707,Oficina,NaN,NaN,0.0,NaN,NaN,Comparación de mercado (directo),8.60,82.34,165503
4,2016-01-12,3.0,NaN,Lima,Lima,Lima,0.0,0.0,-12.089613,-76.994475,Vivienda Unifamiliar,NaN,NaN,43.0,NaN,NaN,Comparación de mercado (directo),357.55,423.22,711009


In [4]:
# Some numerical variables are detected as object
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53254 entries, 0 to 53253
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Fecha entrega del Informe  53254 non-null  datetime64[ns]
 1   Tipo de vía                48243 non-null  float64       
 2   Piso                       21 non-null     object        
 3   Departamento               53254 non-null  object        
 4   Provincia                  53254 non-null  object        
 5   Distrito                   53254 non-null  object        
 6   Número de estacionamiento  45330 non-null  float64       
 7   Depósitos                  45261 non-null  float64       
 8   Latitud (Decimal)          52391 non-null  float64       
 9   Longitud (Decimal)         52337 non-null  float64       
 10  Categoría del bien         50805 non-null  object        
 11  Posición                   113 non-null    object        
 12  Núme

### Reviewing some columns recognized as objects.

'Área Terreno' and 'Valor comercial (USD)' have some commas to be removed in order to get those columns as numeric

In [5]:
Area_terreno = df['Área Terreno'].to_list()
Precio = df['Valor comercial (USD)']
l = [(a, b) for a, b in zip(Area_terreno, Precio)]

In [6]:
l[:30]

[('62.50', 25278),
 ('9.32', 110807),
 ('126.00', 42285),
 ('8.60', 165503),
 ('357.55', 711009),
 ('11.43', 204300),
 ('17.17', 59720),
 ('101.21', 205854),
 ('29.68', 112529),
 ('3.56', 104716),
 ('26.22', 177981),
 ('12.88', 308504),
 ('32.97', 84774),
 ('13.03', 35020),
 ('155.73', 284175),
 ('461.70', 241855),
 ('4.18', 148800),
 ('44.44', 88705),
 ('423.40', 434210),
 ('24.20', 75155),
 ('26.12', 86039),
 ('141,023.00', 76716),
 ('440.00', 429986),
 ('19.26', 190819),
 ('6.71', 78800),
 ('7.06', 211317),
 ('21.00', 157000),
 ('190.00', 250773),
 ('41.18', 258475),
 ('1,158,100.00', 2581207)]

In [7]:
Area_construccion = df['Área Construcción'].to_list()
Precio = df['Valor comercial (USD)']
l2 = [(a, b) for a, b in zip(Area_construccion, Precio)]
l2[:30]

[('27.58', 25278),
 ('57.47', 110807),
 ('84.00', 42285),
 ('82.34', 165503),
 ('423.22', 711009),
 ('90.00', 204300),
 ('59.72', 59720),
 ('425.54', 205854),
 ('77.42', 112529),
 ('55.70', 104716),
 ('110.45', 177981),
 ('168.37', 308504),
 ('86.02', 84774),
 ('34.00', 35020),
 ('291.90', 284175),
 ('232.00', 241855),
 ('80.00', 148800),
 (nan, 88705),
 ('92.50', 434210),
 ('81.69', 75155),
 ('61.75', 86039),
 (nan, 76716),
 ('276.80', 429986),
 ('150.70', 190819),
 ('28.52', 78800),
 ('118.26', 211317),
 ('100.00', 157000),
 ('266.40', 250773),
 ('106.46', 258475),
 ('1,086.00', 2581207)]

### Importing again the data with some forced catagories

In [8]:
df = pd.read_excel('data/dataset_tasacion_train_vf.xlsx',
                  dtype = { # indicate categorical variables
                            'Tipo de vía': 'category',
                            'Piso': 'category',
                            'Departamento': 'category',
                            'Provincia': 'category',
                            'Distrito': 'category',
                            'Categoría del bien': 'category',
                            'Posición': 'category',
                            'Estado de conservación': 'category',
                            'Método Representado': 'category'
                            #,'Área Terreno': np.float64,
                            #'Área Construcción': np.float64
                                

                            }
                  )

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53254 entries, 0 to 53253
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Fecha entrega del Informe  53254 non-null  datetime64[ns]
 1   Tipo de vía                48243 non-null  category      
 2   Piso                       21 non-null     category      
 3   Departamento               53254 non-null  category      
 4   Provincia                  53254 non-null  category      
 5   Distrito                   53254 non-null  category      
 6   Número de estacionamiento  45330 non-null  float64       
 7   Depósitos                  45261 non-null  float64       
 8   Latitud (Decimal)          52391 non-null  float64       
 9   Longitud (Decimal)         52337 non-null  float64       
 10  Categoría del bien         50805 non-null  category      
 11  Posición                   113 non-null    category      
 12  Núme

## Fixing some columns and transform to numeric

In [10]:
df['Área Terreno'] = df['Área Terreno'].apply(lambda x: str(x).replace(',', '')) 
df['Área Construcción'] = df['Área Construcción'].apply(lambda x: str(x).replace(',', '')) 

In [11]:
df['Área Terreno'] = pd.to_numeric(df['Área Terreno'], errors='coerce')
df['Área Construcción'] = pd.to_numeric(df['Área Construcción'], errors='coerce')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53254 entries, 0 to 53253
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Fecha entrega del Informe  53254 non-null  datetime64[ns]
 1   Tipo de vía                48243 non-null  category      
 2   Piso                       21 non-null     category      
 3   Departamento               53254 non-null  category      
 4   Provincia                  53254 non-null  category      
 5   Distrito                   53254 non-null  category      
 6   Número de estacionamiento  45330 non-null  float64       
 7   Depósitos                  45261 non-null  float64       
 8   Latitud (Decimal)          52391 non-null  float64       
 9   Longitud (Decimal)         52337 non-null  float64       
 10  Categoría del bien         50805 non-null  category      
 11  Posición                   113 non-null    category      
 12  Núme

In [13]:
df['Método Representado'].value_counts()

Comparación de mercado (directo)      30243
Costos o reposición (directo)         22970
Renta o capitalización (indirecto)       34
Name: Método Representado, dtype: int64

In [14]:
df.head()

,Fecha entrega del Informe,Tipo de vía,Piso,Departamento,Provincia,Distrito,Número de estacionamiento,Depósitos,Latitud (Decimal),Longitud (Decimal),Categoría del bien,Posición,Número de frentes,Edad,Elevador,Estado de conservación,Método Representado,Área Terreno,Área Construcción,Valor comercial (USD)
0,2020-02-16,1.0,NaN,Piura,Piura,Veintiseis de Octubre,NaN,NaN,-5.163182,-80.682388,Vivienda Unifamiliar,NaN,NaN,0.0,NaN,En construcción,Costos o reposición (directo),62.50,27.58,25278
1,2019-05-21,1.0,NaN,Lima,Lima,Surquillo,1.0,0.0,-12.115286,-77.003768,Departamento,NaN,NaN,1.0,NaN,Bueno,Comparación de mercado (directo),9.32,57.47,110807
2,2020-04-11,1.0,NaN,Junín,Satipo,Satipo,0.0,0.0,-11.235446,-74.629221,Vivienda Unifamiliar,NaN,NaN,7.0,NaN,Regular,Costos o reposición (directo),126.00,84.00,42285
3,2016-05-24,1.0,NaN,Lima,Lima,Santiago de Surco,0.0,0.0,-12.126384,-76.975707,Oficina,NaN,NaN,0.0,NaN,NaN,Comparación de mercado (directo),8.60,82.34,165503
4,2016-01-12,3.0,NaN,Lima,Lima,Lima,0.0,0.0,-12.089613,-76.994475,Vivienda Unifamiliar,NaN,NaN,43.0,NaN,NaN,Comparación de mercado (directo),357.55,423.22,711009


In [15]:
df.shape

(53254, 20)

In [16]:
df.columns

Index(['Fecha entrega del Informe', 'Tipo de vía', 'Piso', 'Departamento',
       'Provincia', 'Distrito', 'Número de estacionamiento', 'Depósitos',
       'Latitud (Decimal)', 'Longitud (Decimal)', 'Categoría del bien',
       'Posición', 'Número de frentes', 'Edad', 'Elevador',
       'Estado de conservación', 'Método Representado', 'Área Terreno',
       'Área Construcción', 'Valor comercial (USD)'],
      dtype='object')

In [17]:
df['Categoría del bien'].unique()

['Vivienda Unifamiliar', 'Departamento', 'Oficina', 'Local Comercial', 'AVALUOS_TIPOS_INMUEBLE_VEHICULO', ..., 'Intitución Educativa', 'Fundo Agrícola', 'Terreno Rústico (eriazo)', 'Muebles y enseres', 'Maquinaria y/o Equipo']
Length: 17
Categories (16, object): ['AVALUOS_TIPOS_INMUEBLE_VEHICULO', 'Almacén /Taller', 'Centro de Salud', 'Departamento', ..., 'Oficina', 'Terreno Rústico (eriazo)', 'Terreno Urbano', 'Vivienda Unifamiliar']

In [18]:
df['Categoría del bien'].value_counts()

Departamento                       22347
Vivienda Unifamiliar               10966
Local Comercial                     5578
Terreno Urbano                      5089
AVALUOS_TIPOS_INMUEBLE_VEHICULO     1907
Almacén /Taller                     1161
Industria                           1127
Oficina                             1114
Estacionamiento/depósito (U.I.)      870
Hotel                                319
Intitución Educativa                 144
Centro de Salud                       99
Fundo Agrícola                        48
Terreno Rústico (eriazo)              34
Maquinaria y/o Equipo                  1
Muebles y enseres                      1
Name: Categoría del bien, dtype: int64

### Exploring number of observations for different Categories

In [19]:
dist_cat = df['Categoría del bien'].value_counts(normalize=True)*100
dist_cat = pd.DataFrame(dist_cat)
dist_cat

,Categoría del bien
Departamento,43.985828
Vivienda Unifamiliar,21.584490
Local Comercial,10.979234
Terreno Urbano,10.016731
AVALUOS_TIPOS_INMUEBLE_VEHICULO,3.753568
Almacén /Taller,2.285208
Industria,2.218286
Oficina,2.192698
Estacionamiento/depósito (U.I.),1.712430
Hotel,0.627891


In [20]:
categorias = df['Categoría del bien'].value_counts().keys()
categorias

CategoricalIndex(['Departamento', 'Vivienda Unifamiliar', 'Local Comercial',
                  'Terreno Urbano', 'AVALUOS_TIPOS_INMUEBLE_VEHICULO',
                  'Almacén /Taller', 'Industria', 'Oficina',
                  'Estacionamiento/depósito (U.I.)', 'Hotel',
                  'Intitución Educativa', 'Centro de Salud', 'Fundo Agrícola',
                  'Terreno Rústico (eriazo)', 'Maquinaria y/o Equipo',
                  'Muebles y enseres'],
                 categories=['AVALUOS_TIPOS_INMUEBLE_VEHICULO', 'Almacén /Taller', 'Centro de Salud', 'Departamento', 'Estacionamiento/depósito (U.I.)', 'Fundo Agrícola', 'Hotel', 'Industria', ...], ordered=False, dtype='category')

In [ ]:
plt.figure(figsize=(20,8))
df['Categoría del bien'].value_counts().plot(kind='bar')
plt.title('Distribución de observaciones por Categoría del bien')
plt.ylabel('Número de observaciones')
plt.xlabel('Categoria del bien')
plt.savefig('Distribucion.png', bbox_inches='tight', pad_inches=0.1)

### Missing Values

In [ ]:
missing = (df.isnull().sum()/len(df)*100).sort_values(ascending=False).reset_index()
missing  = pd.DataFrame(missing)
missing.set_axis(['Feature', '% missing'], axis=1, inplace=True)
missing

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# this particular feature has too little observations and the ones registerd are not well codified
df['Piso'].value_counts()

In [ ]:
df['Elevador'].value_counts()

In [ ]:
# Exploring some individual columns we can detect wrong values that must be thresholded
df['Número de frentes'].value_counts()

In [ ]:
df['Posición'].value_counts()

### Plotting features vs Price

In [ ]:
varstolook = df.columns.to_list()
varstolook.remove('Fecha entrega del Informe')
varstolook.remove('Valor comercial (USD)')
varstolook.remove('Distrito')
varstolook.remove('Provincia')
varstolook.remove('Piso')

varstolook

In [ ]:
target = 'Valor comercial (USD)'
def figures_grid(nfigsx, nfigsy, figs_functions, figsize=None):
    if figsize is None:
        figsize = (nfigsx*3, nfigsy*3)
        
    fig = plt.figure(figsize=figsize)
    i = 1
    for y in range(1, nfigsy+1):
        for x in range(1,nfigsx+1):
            if i<=len(figs_functions):
                axis = fig.add_subplot(nfigsy, nfigsx, i)
                figs_functions[i-1]()
            i+=1


def plot_vars(col, target):
    
    def f1(): 
        if pd.api.types.is_categorical_dtype(df[col]):
            k = df[col].value_counts()
            sns.barplot(x=k.index, y=k.values)
        else:
            sns.distplot(df[col].dropna())
        plt.title("distribution of %s"%col)
        plt.grid();
        
    def f2(): 
        if pd.api.types.is_categorical_dtype(df[col]):
            k=df[[col,target]].dropna()
            for v in df[col].dropna().cat.categories:
                if sum(k[col]==v)>1:
                    sns.distplot(k[target][k[col]==v], 
                                 hist_kws=dict(alpha=.3), 
                                 kde_kws=dict(linewidth=1, alpha=.8),
                                 label=v);
            if sum(df[col].isna())>1:
                sns.distplot(df[target][df[col].isna()], 
                             hist_kws=dict(alpha=.8), 
                             kde_kws=dict(linewidth=1, alpha=1),
                             label="missing");
            plt.legend();
        else:
            plt.scatter(df[col], df[target], alpha=.5)
            plt.xlabel(target)
            plt.ylabel(col)
        plt.grid()
        plt.title("%s vs %s"%(col, target));
        
    def f3():
        if pd.api.types.is_categorical_dtype(df[col]):
            sns.violinplot(x=col, y=target, data=df)
            plt.title("Violin plot of distribution of %s wrt %s"%(target, col))
            #plt.yticks([])
            plt.grid()
        else:
            # in case of continuous numerical columns the variable is binned in ranges to be able to create a clear violin plot
            ranges = np.arange(df[col].min(), df[col].max(), (df[col].max()-df[col].min())/5)
            sns.violinplot(x=pd.cut(df[col], ranges), y=target, data=df)
            plt.title("Violin plot of distribution of %s wrt %s binned by ranges"%(target, col))
            #plt.yticks([])
            plt.grid()
        
        
    figures_grid(3,1, [f1, f2, f3], figsize=(25,5));      


for col in varstolook:
    plot_vars(col, target=target);

In [ ]:
df2 = df.copy()

In [ ]:
def figures_grid(nfigsx, nfigsy, figs_functions, figsize=None):
    if figsize is None:
        figsize = (nfigsx*3, nfigsy*3)
        
    fig = plt.figure(figsize=figsize)
    i = 1
    for y in range(1, nfigsy+1):
        for x in range(1,nfigsx+1):
            if i<=len(figs_functions):
                axis = fig.add_subplot(nfigsy, nfigsx, i)
                figs_functions[i-1]()
            i+=1


def plot_vars(col, target):
    
    def f1(): 
        if pd.api.types.is_categorical_dtype(df2[col]):
            k = df2[col].value_counts()
            sns.barplot(x=k.index, y=k.values)
        else:
            sns.distplot(df2[col].dropna())
        plt.title("distribution of %s"%col)
        plt.grid();
        
    def f2(): 
        if pd.api.types.is_categorical_dtype(df2[col]):
            k=df2[[col,target]].dropna()
            for v in df2[col].dropna().cat.categories:
                if sum(k[col]==v)>1:
                    sns.distplot(k[target][k[col]==v], 
                                 hist_kws=dict(alpha=.3), 
                                 kde_kws=dict(linewidth=1, alpha=.8),
                                 label=v);
            if sum(df2[col].isna())>1:
                sns.distplot(df2[target][df2[col].isna()], 
                             hist_kws=dict(alpha=.8), 
                             kde_kws=dict(linewidth=1, alpha=1),
                             label="missing");
            plt.legend();
        else:
            plt.scatter(df2[col], df2[target], alpha=.5)
            plt.xlabel(target)
            plt.ylabel(col)
        plt.grid()
        plt.title("%s vs %s"%(col, target));
        
    def f3():
        if pd.api.types.is_categorical_dtype(df2[col]):
            sns.violinplot(x=col, y=target, data=df)
            plt.title("Violin plot of distribution of %s wrt %s"%(target, col))
            #plt.yticks([])
            plt.grid()
        else:
            # in case of continuous numerical columns the variable is binned in ranges to be able to create a clear violin plot
            ranges = np.arange(df2[col].min(), df2[col].max(), (df2[col].max()-df2[col].min())/5)
            sns.violinplot(x=pd.cut(df2[col], ranges), y=target, data=df2)
            plt.title("Violin plot of distribution of %s wrt %s binned by ranges"%(target, col))
            #plt.yticks([])
            plt.grid()
        
        
    figures_grid(3,1, [f1, f2, f3], figsize=(25,5));      


for col in varstolook:
    plot_vars(col, target=target);

### Correlation matrix

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), cmap="RdYlBu", 
    annot=True, square=True,
    vmin=-0.8, vmax=0.8, fmt="+.1f")
plt.title("Correlations between predictors");